In [95]:
import tensorflow as tf
import os
import pathlib
import shutil


In [96]:
 #Clear previous directories if they exist to avoid any conflicts
data_dir = pathlib.Path('/root/.keras/datasets/cats_and_dogs_filtered')
if os.path.exists(data_dir):
    print("Found existing directory, clearing it...")
    shutil.rmtree(data_dir)

Found existing directory, clearing it...


In [97]:
train_dir = data_dir / 'train'
validation_dir = data_dir / 'validation'

train_cats_dir = train_dir / 'cats'
train_dogs_dir = train_dir / 'dogs'
validation_cats_dir = validation_dir / 'cats'
validation_dogs_dir = validation_dir / 'dogs'

In [98]:
os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(validation_cats_dir, exist_ok=True)
os.makedirs(validation_dogs_dir, exist_ok=True)

In [99]:
source_dir = pathlib.Path('/root/.keras/datasets/cats_and_dogs_filtered_extracted/cats_and_dogs_filtered')


In [100]:
print("Copying files...")
shutil.copytree(str(source_dir / 'train' / 'cats'), str(train_cats_dir), dirs_exist_ok=True)
shutil.copytree(str(source_dir / 'train' / 'dogs'), str(train_dogs_dir), dirs_exist_ok=True)
shutil.copytree(str(source_dir / 'validation' / 'cats'), str(validation_cats_dir), dirs_exist_ok=True)
shutil.copytree(str(source_dir / 'validation' / 'dogs'), str(validation_dogs_dir), dirs_exist_ok=True)

Copying files...


'/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs'

In [101]:
print("\nNumber of images in each directory:")
print("Training cats:", len(os.listdir(train_cats_dir)))
print("Training dogs:", len(os.listdir(train_dogs_dir)))
print("Validation cats:", len(os.listdir(validation_cats_dir)))
print("Validation dogs:", len(os.listdir(validation_dogs_dir)))


Number of images in each directory:
Training cats: 1000
Training dogs: 1000
Validation cats: 500
Validation dogs: 500


In [102]:
print(train_cats_dir)

/root/.keras/datasets/cats_and_dogs_filtered/train/cats


In [103]:
total_train = len(os.listdir(train_cats_dir))+ len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(validation_cats_dir)) + len(os.listdir(validation_dogs_dir))

In [104]:
from tensorflow.keras.preprocessing.image\
import ImageDataGenerator

In [105]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [106]:
batch_size = 16
img_height = 100
img_width = 100

In [107]:
train_data_gen = train_image_generator.flow_from_directory\
                 (batch_size=batch_size, directory=train_dir, \
                  shuffle=True, \
                  target_size=(img_height, img_width), \
 class_mode='binary')

Found 2000 images belonging to 2 classes.


In [108]:
val_data_gen = validation_image_generator.flow_from_directory\
              (batch_size=batch_size, \
               directory=validation_dir, \
               target_size=(img_height, img_width), class_mode='binary')

Found 1000 images belonging to 2 classes.


In [109]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [110]:
np.random.seed(8)
tf.random.set_seed(8)

In [111]:
model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', \
                  input_shape=(img_height, img_width ,3)),\
    layers.MaxPooling2D(),\
    layers.Conv2D(128, 3, activation='relu'),\
    layers.MaxPooling2D(),\
    layers.Flatten(),\
    layers.Dense(128, activation='relu'),\
    layers.Dense(1, activation='sigmoid')])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [112]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [113]:
model.compile(loss='binary_crossentropy', \
              optimizer=optimizer, metrics=['accuracy'])

In [114]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 98, 98, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 49, 49, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 47, 47, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 67712)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       8,667,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,743,041 (33.35 MB)

 Trainable params: 8,743,041 (33.35 MB)

 Non-trainable params: 0 (0.00 B)

In [115]:
model.fit(train_data_gen,
          steps_per_epoch=total_train // batch_size,
          epochs=5,
          validation_data=val_data_gen,
          validation_steps=total_val // batch_size)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 113s 885ms/step - accuracy: 0.5235 - loss: 0.7881 - val_accuracy: 0.5040 - val_loss: 0.6896
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 133s 821ms/step - accuracy: 0.5967 - loss: 0.6778 - val_accuracy: 0.6280 - val_loss: 0.6594
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 819ms/step - accuracy: 0.6866 - loss: 0.6095 - val_accuracy: 0.6532 - val_loss: 0.6614
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 141s 805ms/step - accuracy: 0.7654 - loss: 0.4803 - val_accuracy: 0.6492 - val_loss: 0.6412
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.8364 - loss: 0.3541 - val_accuracy: 0.6562 - val_loss: 0.7810
